# **Dependancies**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import *
from keras.callbacks import *
import os
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from commons import mean_absolute_percentage_error
from keras.layers import *
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.losses import LogCosh

# **Loading Data**

In [4]:
PATH_TO_DATA = "data/reg_interval20-23wma.csv"
data = pd.read_csv(PATH_TO_DATA)

In [5]:
data.head()

,difficulty3rsi,mediantransactionvalue30trxUSD,mining_profitability90trx,price30momUSD,price30varUSD,price3momUSD,price3varUSD,price90momUSD,price90varUSD,top100cap3trx,transactionvalueUSD,price3wmaUSD
0,100.0,-0.214,-0.320,1477.0,185824.0,484.627,46350.0,523.430,586279.0,-0.016,30967.0,8616.0
1,100.0,-0.184,-0.321,1694.0,231148.0,632.769,70029.0,721.647,593953.0,-0.019,42231.0,8707.0
2,100.0,-0.157,-0.321,1840.0,253780.0,542.568,5019.0,713.823,600131.0,-0.044,23647.0,8795.0
3,100.0,-0.128,-0.321,2070.0,278575.0,263.765,5124.0,889.842,609381.0,-0.054,30527.0,8859.0
4,100.0,-0.108,-0.321,1729.0,316028.0,128.570,8649.0,876.998,619179.0,-0.041,15636.0,8877.0


In [6]:
data.shape

(639, 12)

In [7]:
y = data['price3wmaUSD']

In [8]:
X = data

In [9]:
X['price3wmaUSD'] = X['price3wmaUSD'].shift(1,fill_value=4000)
#X=X[X['priceUSD']!=1]

In [10]:
X

,difficulty3rsi,mediantransactionvalue30trxUSD,mining_profitability90trx,price30momUSD,price30varUSD,price3momUSD,price3varUSD,price90momUSD,price90varUSD,top100cap3trx,transactionvalueUSD,price3wmaUSD
0,100.000,-0.214,-0.320,1477.000,185824.0,484.627,46350.000,523.430,586279.0,-0.016,30967.0,4000.0
1,100.000,-0.184,-0.321,1694.000,231148.0,632.769,70029.000,721.647,593953.0,-0.019,42231.0,8616.0
2,100.000,-0.157,-0.321,1840.000,253780.0,542.568,5019.000,713.823,600131.0,-0.044,23647.0,8707.0
3,100.000,-0.128,-0.321,2070.000,278575.0,263.765,5124.000,889.842,609381.0,-0.054,30527.0,8795.0
4,100.000,-0.108,-0.321,1729.000,316028.0,128.570,8649.000,876.998,619179.0,-0.041,15636.0,8859.0
...,...,...,...,...,...,...,...,...,...,...,...,...
634,99.043,-0.085,-0.406,425.550,82135.0,-179.890,8243.000,-2758.000,2474152.0,0.031,53190.0,16731.0
635,99.043,-0.087,-0.405,177.977,76543.0,-258.919,8530.000,-2936.000,2479394.0,0.047,54195.0,16643.0
636,99.043,-0.090,-0.404,-347.725,82274.0,-257.127,1350.000,-2778.000,2489811.0,0.027,50405.0,16579.0
637,99.043,-0.094,-0.403,-489.020,86919.0,-69.693,287.999,-2661.000,2500289.0,0.030,42790.0,16567.0


In [11]:
#X1 = data
#X1['target'] = X1['priceUSD'].shift(-1)


In [12]:
#X1.dropna(subset=['target'], inplace=True)

In [13]:
#X = data.iloc[:,:-1]

In [14]:
#y=data.iloc[:,-1:]

In [15]:
#y = data['priceUSD'].shift(-1,fill_value=1)

In [16]:
#y = data['priceUSD']

In [17]:
y

0       8616.0
1       8707.0
2       8795.0
3       8859.0
4       8877.0
        ...   
634    16643.0
635    16579.0
636    16567.0
637    16561.0
638    16627.0
Name: price3wmaUSD, Length: 639, dtype: float64

In [18]:
def create_dataset(dataset, time_step=1):
    dataX = []
    datay = []
    for i in range(len(dataset) - time_step):
        a = dataset.iloc[i:(i + time_step), :].values  # Use .iloc for DataFrame
        dataX.append(a)
        datay.append(dataset.iloc[i + time_step, -1])
    return np.array(dataX), np.array(datay)


In [19]:
#time_step = 15
#X2 ,y2 = create_dataset(X1, time_step)

#print("X: ", X2.shape)
#print("y: ", y2.shape)

In [20]:
#X2

In [21]:
#y2

In [22]:
#y2_reshaped = y2.reshape(-1, 1, 1)

In [23]:
#y2_reshaped

In [24]:
X_train, X_test, y_train, y_test =train_test_split(X,y, test_size=0.2, train_size=0.8, shuffle=False, random_state=8)

In [25]:
X_train.shape

(511, 12)

In [26]:
estimators=[]

In [27]:
estimators.append(['robust',RobustScaler()])

In [28]:
estimators.append(['mixmax',MinMaxScaler()])

In [29]:
scale=Pipeline(estimators,verbose=True)

In [30]:
X_train=scale.fit_transform(X_train)

[Pipeline] ............ (step 1 of 2) Processing robust, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing mixmax, total=   0.0s


In [31]:
X_test=scale.transform(X_test)

In [32]:
X_train=np.reshape(X_train,(X_train.shape[0],1,X_train.shape[1]))

In [33]:
X_test=np.reshape(X_test,(X_test.shape[0],1,X_test.shape[1]))

In [34]:
y_train=y_train.values

In [35]:
y_train=np.reshape(y_train, (y_train.shape[0],1,1))

In [36]:
y_test=y_test.values

In [37]:
y_test=np.reshape(y_test,(y_test.shape[0],1,1))

In [38]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [39]:
X_train.shape

(511, 1, 12)

In [40]:
X_train

array([[[1.0000000e+00, 2.0612398e-01, 8.4264830e-02, ...,
         3.1855136e-01, 7.9214415e-03, 0.0000000e+00]],

       [[1.0000000e+00, 2.2852875e-01, 8.3404988e-02, ...,
         3.1633407e-01, 1.3735418e-02, 1.0250944e-01]],

       [[1.0000000e+00, 2.4869305e-01, 8.3404988e-02, ...,
         2.9785663e-01, 4.1431817e-03, 1.0453031e-01]],

       ...,

       [[9.9999997e-06, 2.1732636e-01, 5.1590712e-03, ...,
         3.2224685e-01, 2.0522813e-01, 3.7492782e-01]],

       [[9.9999997e-06, 2.1433906e-01, 2.5795356e-03, ...,
         3.5772356e-01, 1.6259265e-01, 3.6031535e-01]],

       [[9.9999997e-06, 2.1359223e-01, 0.0000000e+00, ...,
         3.5181078e-01, 2.3403892e-01, 3.5087720e-01]]], dtype=float32)

In [41]:
y_train.shape

(511, 1, 1)

In [42]:
y_train

array([[[ 8616.]],

       [[ 8707.]],

       [[ 8795.]],

       [[ 8859.]],

       [[ 8877.]],

       [[ 8773.]],

       [[ 8703.]],

       [[ 8682.]],

       [[ 8577.]],

       [[ 8477.]],

       [[ 8586.]],

       [[ 8850.]],

       [[ 9148.]],

       [[ 9322.]],

       [[ 9373.]],

       [[ 9384.]],

       [[ 9391.]],

       [[ 9376.]],

       [[ 9301.]],

       [[ 9347.]],

       [[ 9532.]],

       [[ 9704.]],

       [[ 9795.]],

       [[ 9948.]],

       [[ 9964.]],

       [[ 9975.]],

       [[10143.]],

       [[10249.]],

       [[10287.]],

       [[10215.]],

       [[10054.]],

       [[ 9859.]],

       [[ 9814.]],

       [[ 9946.]],

       [[ 9810.]],

       [[ 9730.]],

       [[ 9664.]],

       [[ 9761.]],

       [[ 9772.]],

       [[ 9653.]],

       [[ 9332.]],

       [[ 9010.]],

       [[ 8798.]],

       [[ 8714.]],

       [[ 8634.]],

       [[ 8683.]],

       [[ 8742.]],

       [[ 8773.]],

       [[ 8900.]],

       [[ 9019.]],



In [43]:
#X_train, X_test, y_train, y_test =train_test_split(X2,y2_reshaped, test_size=0.2, train_size=0.8, shuffle=False, random_state=8)

In [44]:
#X_train.shape[2]

In [45]:
#y_train.shape

In [46]:
#X_test.shape

# **Model Architecture + Training**

In [47]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [48]:

adam = optimizers.Adam(learning_rate=lr_schedule(0), amsgrad=True)

Learning rate:  0.001


In [49]:
from tensorflow.keras import backend as K

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [50]:
model = Sequential()
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu'), input_shape=(1, X_train.shape[2])))
model.add(Bidirectional(LSTM(350, return_sequences=True, activation='relu')))
model.add(Dense(1))
model.compile(loss="log_cosh", optimizer=adam, metrics=['mae'])

E:\DS440proj\BTCpred\.venv\lib\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [51]:
mcp_save = ModelCheckpoint('LSTM_reg_18-21rmse355.keras', save_best_only=True, monitor='val_loss', mode='auto')
earlyStopping = EarlyStopping(monitor='val_loss', patience=100, verbose=1, mode='auto')

In [52]:
#model.compile(optimizer='adam', loss='log_cosh', metrics=['accuracy'])
model.fit(X_train,y_train, epochs=5000, batch_size=32, validation_data=(X_test,y_test), callbacks=[mcp_save,earlyStopping])

Epoch 1/5000
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 26910.4492 - mae: 26911.1406 - val_loss: 18850.5273 - val_mae: 18851.2227
Epoch 2/5000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 27350.0781 - mae: 27350.7695 - val_loss: 18843.4746 - val_mae: 18844.1680
Epoch 3/5000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 25879.0762 - mae: 25879.7676 - val_loss: 18793.9590 - val_mae: 18794.6523
Epoch 4/5000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 26135.6816 - mae: 26136.3711 - val_loss: 18635.7109 - val_mae: 18636.4043
Epoch 5/5000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 26689.5352 - mae: 26690.2246 - val_loss: 18312.5918 - val_mae: 18313.2832
Epoch 6/5000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 26901.8750 - mae: 26902.5645 - val_loss: 17698.9844 - val_mae: 17699.6797
Epoch 7/5000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - loss: 24866.5098 - mae: 24867.2031 - val_loss: 16624.5879 - val_mae: 16625.2832
Epoch 8/5000
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - 

# **Testing Model**

In [53]:
#model.save('LSTM_complete_model344.h5', overwrite=True)

In [54]:
#from tensorflow.keras.models import load_model
#loaded_model = load_model('LSTM_complete_model344.h5')

In [55]:
y_pred = np.ravel(model.predict(X_test))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  


In [56]:
y_test=np.ravel(y_test)

In [57]:
y_pred

array([19858.191, 20646.291, 20984.748, 21235.31 , 21299.861, 21013.057,
       20823.135, 21150.578, 21340.031, 21330.9  , 21140.402, 20305.383,
       19696.863, 19624.488, 19862.545, 20044.982, 19943.525, 19922.396,
       19652.943, 19658.479, 19656.328, 19648.236, 19064.178, 19016.865,
       20001.602, 21011.85 , 21166.98 , 20585.477, 19797.648, 19367.979,
       19563.605, 19642.004, 19287.393, 19016.805, 18837.623, 18873.127,
       18834.574, 18860.82 , 18857.117, 18897.709, 19358.252, 19187.738,
       19243.99 , 19130.193, 19090.475, 18875.576, 18925.404, 19417.557,
       19740.3  , 19841.73 , 19544.973, 19197.71 , 19009.326, 19085.848,
       18934.295, 18849.795, 18809.984, 19162.723, 19001.586, 18994.273,
       19030.705, 19264.564, 19167.234, 18949.979, 18828.68 , 18892.367,
       19001.246, 19136.762, 19326.121, 20058.518, 20454.88 , 20391.596,
       20448.764, 20458.77 , 20442.727, 20309.736, 20265.34 , 20188.426,
       20360.889, 20780.746, 21180.31 , 20772.547, 

In [58]:
r2=r2_score(y_test,y_pred) #testing score/ r^2
r2

0.9594395957925618

In [59]:
mae=mean_absolute_error(y_test,y_pred) #mae
mae

272.07953

In [60]:
rmse=np.sqrt(mean_squared_error(y_test,y_pred)) #rmse
rmse

325.80548

In [61]:
mape=mean_absolute_percentage_error(y_test,y_pred) #mape
mape

1.4558171853423119

In [62]:
pd.DataFrame(zip(['MAE','RMSE','MAPE','R^2'],[mae,rmse,mape,r2])).transpose()

,0,1,2,3
0,MAE,RMSE,MAPE,R^2
1,272.079529,325.805481,1.455817,0.95944


In [63]:
pd.DataFrame([y_test,y_pred]).transpose()

,0,1
0,20382.0,19858.191406
1,20722.0,20646.291016
2,21030.0,20984.748047
3,21455.0,21235.310547
4,21490.0,21299.861328
...,...,...
123,16643.0,16530.423828
124,16579.0,16432.117188
125,16567.0,16351.436523
126,16561.0,16362.191406
